source : https://medium.com/geekculture/different-ways-to-calculate-cosine-similarity-in-python-ae5bb28c372c
source : https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1
source : https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/#1-loading-pre-trained-bert


In [ ]:
import numpy as np
import pandas as pd

# Cosine similarity - sklearn

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

print("cosine similarity :",cosine_similarity(trsfm[0], trsfm)[0][1])


# BERT Word Embeddings Tutorial

## Loading Pre-Trained BERT


In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# BERT For Measuring Text Similarity


In [ ]:
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/NLI_contradiction_my_dear_watson/data/translated_data_train.csv")

In [ ]:
labels = {0 : "entailment",
          1 : "neutral",
          2 : "contradiction"}
df_train["label_str"] = df_train["label"].apply(lambda l : labels[l])

In [ ]:
df_train["both_sentences"] = df_train["premise"].astype(str) + "[SEP]" + df_train["hypothesis"]
df_train["both_sentences"] = df_train["both_sentences"].apply(lambda s : s.split("[SEP]"))

In [ ]:
df_train.head(2)

In [ ]:
len(df_train)

In [ ]:
train_data_lang = df_train.groupby('language').count().reset_index()[['language', 'id']]

train_data_lang['sample_count'] = df_train['id'] / 10

# sample data
sample_train_data = pd.DataFrame(columns=df_train.columns)
for i in range(len(train_data_lang)):
    df = df_train[df_train['language'] == train_data_lang['language'][i]]
    n = int(train_data_lang['sample_count'][i])
    df = df.sample(n).reset_index(drop=True)
    sample_train_data = sample_train_data.append(df)
    sample_train_data = sample_train_data.reset_index(drop=True)

In [ ]:
import time
avg_elapsed = 0
i = 0
t_total = time.time()
sentence_embedding = []
for v in df_train["both_sentences"].values :
    t = time.time()
    sentence_embedding.append(model.encode(v))
    i += 1
    avg_elapsed = (avg_elapsed * (i-1) + (time.time() - t)) / i
    if i % 100 == 0 :
        print(str(i).rjust(6),"/",12120)
        print("avg_elapsed :",np.round(avg_elapsed,2),"s. - ETA :",np.round(avg_elapsed * (12120-i),2),"s. - elapsed :",np.round(time.time()-t_total,2),"s.\n")
# df_train["sentence_embedding"] = df_train["both_sentences"].apply(model.encode)

In [ ]:
df_train["sentence_embedding"] = sentence_embedding
df_train.head(5)

In [ ]:
df_train.to_csv("/content/drive/MyDrive/NLI_contradiction_my_dear_watson/data/df_translated_train.csv")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
df_train["cosine_similarity"] = df_train["sentence_embedding"].apply(lambda x : cosine_similarity(x[0].reshape(1,-1),x[1].reshape(1,-1))[0][0])

In [ ]:
df_train.head(5)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:


plt.figure(figsize=(10,3))
plt.title("representation of the cosine similarity\nThe y axis repsresents only the label for better visualization.")
sns.scatterplot(x=df_train["cosine_similarity"], y=df_train["label"], hue=df_train["label_str"])
plt.legend()
plt.plot()